In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from utils import preprocessing, transforms
from utils.constants import PLOTCOLORS
import matplotlib.pyplot as plt
from scipy import stats

RANDOM_STATE = 21

cwd = Path.cwd()
husky_data = cwd / "data" / "borealtc"
vulpi_data = cwd / "data" / "vulpi"
figs_path = cwd / "analysis" / "figs"
figs_path.mkdir(parents=True, exist_ok=True)

COLORS = {
    "base": PLOTCOLORS["dirt road"],
    "ours": PLOTCOLORS["concrete"],
}

In [ ]:
# Define channels
columns = {
    "imu": {
        "wx": True,
        "wy": True,
        "wz": True,
        "ax": True,
        "ay": True,
        "az": True,
    },
    "pro": {
        "velL": True,
        "velR": True,
        "curL": True,
        "curR": True,
    },
}
summary = pd.DataFrame({"columns": pd.Series(columns)})

In [ ]:
husky_summ = summary.copy()
vulpi_summ = summary.copy()

husky_dfs = preprocessing.get_recordings(husky_data, husky_summ)
vulpi_dfs = preprocessing.get_recordings(vulpi_data, vulpi_summ)

husky_pro = husky_dfs["pro"].copy()
vulpi_pro = vulpi_dfs["pro"].copy()

In [ ]:
husky_pro["vx"], husky_pro["wz"] = transforms.unicycle_model(husky_pro)
vulpi_pro["vx"], vulpi_pro["wz"] = transforms.unicycle_model(vulpi_pro)

In [ ]:
with plt.style.context("./analysis/figure.mplstyle"):
    fig, axs = plt.subplots(ncols=2)
    vcols = [COLORS["base"], COLORS["ours"]]
    v_vx = axs[0].violinplot([vulpi_pro.vx, husky_pro.vx], showmeans=False)
    v_wz = axs[1].violinplot([vulpi_pro.wz, husky_pro.wz], showmeans=False)

    for pc_vx, pc_wz, col in zip(v_vx["bodies"], v_wz["bodies"], vcols):
        pc_vx.set_facecolor(col)
        pc_wz.set_facecolor(col)
        pc_vx.set_edgecolor(col)
        pc_wz.set_edgecolor(col)
        pc_vx.set_alpha(1)
        pc_wz.set_alpha(1)

    axs[0].set_xticks([1, 2], ["Baseline", "Ours"])
    axs[1].set_xticks([1, 2], ["Baseline", "Ours"])
    axs[0].set_ylim((-0.9, 0.9))
    axs[1].set_ylim((-0.9, 0.9))

    axs[0].set_ylabel("Linear speed $v_x$ (m/s)")
    axs[1].set_ylabel("Yaw rate $\omega_z$ (rad/s)")

    fig.tight_layout()
    plt.show()

In [ ]:
# with plt.style.context('./analysis/figure.mplstyle'):
fig, axs = plt.subplots(ncols=2)
vcols = [COLORS["base"], COLORS["ours"]]
plt_terrains = ["SANDY_LOAM", "SNOW"]
lab_terrains = [terr.replace("_", " ").title() for terr in plt_terrains]
col_terrains = [PLOTCOLORS[lab.lower()] for lab in lab_terrains]
xpos = [*range(len(plt_terrains))]

vx = [husky_pro[husky_pro.terrain == terr].vx for terr in plt_terrains]
wz = [husky_pro[husky_pro.terrain == terr].wz for terr in plt_terrains]

v_vx = axs[0].violinplot(vx, positions=xpos, showmeans=True)
v_wz = axs[1].violinplot(wz, positions=xpos, showmeans=True)

for pc_vx, pc_wz, col in zip(v_vx["bodies"], v_wz["bodies"], col_terrains):
    pc_vx.set_facecolor(col)
    pc_wz.set_facecolor(col)
    pc_vx.set_edgecolor(col)
    pc_wz.set_edgecolor(col)
    pc_vx.set_alpha(1)
    pc_wz.set_alpha(1)

axs[0].set_xticks(xpos, lab_terrains)
axs[1].set_xticks(xpos, lab_terrains)
# axs[0].set_ylim((-0.25, 0.75))
# axs[1].set_ylim((-0.25, 0.75))

axs[0].set_ylabel("Linear speed $v_x$ (m/s)")
axs[1].set_ylabel("Yaw rate $\omega_z$ (rad/s)")

fig.tight_layout()
plt.show()

In [ ]:
with plt.style.context("./analysis/figure.mplstyle"):
    fig = plt.figure(figsize=(9, 17))
    subfigs = fig.subfigures(nrows=2)

    ax = subfigs[0].subplots()
    ax.scatter(
        vulpi_pro.wz,
        vulpi_pro.vx,
        c=COLORS["base"],
        s=0.5,
        alpha=1,
        label="Baseline",
        zorder=5,
    )
    ax.scatter(
        husky_pro.wz,
        husky_pro.vx,
        c=COLORS["ours"],
        s=0.5,
        alpha=0.8,
        label="Ours",
        zorder=-1,
    )
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), markerscale=10)
    ax.set_xlabel("Yaw rate $\omega_z$ (rad/s)")
    ax.set_ylabel("Linear speed $v_x$ (m/s)")
    ax.set_aspect("equal")
    ax.set_xlim((-0.8, 0.8))
    ax.set_ylim((-0.8, 0.8))

    axs = subfigs[1].subplots(ncols=2, width_ratios=[1, 1])
    vcols = [COLORS["base"], COLORS["ours"]]
    v_vx = axs[0].violinplot([vulpi_pro.vx, husky_pro.vx], showmeans=False)
    v_wz = axs[1].violinplot([vulpi_pro.wz, husky_pro.wz], showmeans=False)

    for pc_vx, pc_wz, col in zip(v_vx["bodies"], v_wz["bodies"], vcols):
        pc_vx.set_facecolor(col)
        pc_wz.set_facecolor(col)
        pc_vx.set_edgecolor(col)
        pc_wz.set_edgecolor(col)
        pc_vx.set_alpha(1)
        pc_wz.set_alpha(1)

    axs[0].set_xticks([1, 2], ["Baseline", "Ours"])
    axs[1].set_xticks([1, 2], ["Baseline", "Ours"])
    axs[0].set_ylim((-0.9, 0.9))
    axs[1].set_ylim((-0.9, 0.9))

    axs[0].set_ylabel("Linear speed $v_x$ (m/s)")
    axs[1].set_ylabel("Yaw rate $\omega_z$ (rad/s)")

fig.tight_layout()
plt.show()

# Rotationnality

In [ ]:
from utils.constants import HuskyConstants

Bs = HuskyConstants.ugv_Bs
husky_pro["rho"] = (
    Bs * husky_pro.wz.abs() / (husky_pro.vx.abs() + Bs * husky_pro.wz.abs())
)
vulpi_pro["rho"] = (
    Bs * vulpi_pro.wz.abs() / (vulpi_pro.vx.abs() + Bs * vulpi_pro.wz.abs())
)

In [ ]:
fig, ax = plt.subplots()

terrains = sorted(husky_pro.terrain.unique())
rhos = [husky_pro[husky_pro.terrain == terr].rho for terr in terrains]
labels = [lab.replace("_", " ").title() for lab in terrains]
cols = [PLOTCOLORS[lab.lower()] for lab in labels]

ax.hist(
    rhos,
    label=labels,
    color=cols,
    log=True,
)
ax.legend(loc="center left", bbox_to_anchor=(1.05, 0.5))
ax.set_xlabel("Rotationnality $\\rho$")
ax.set_ylabel("Counts")
ticks = np.round(0.1 * np.arange(11), 1)
ax.set_xticks(ticks, ticks)
plt.show()

In [ ]:
rho_desc = {
    "ours": stats.describe(husky_pro.rho.dropna())._asdict(),
    "baseline": stats.describe(vulpi_pro.rho.dropna())._asdict(),
}
pd.DataFrame.from_dict(rho_desc, orient="index")

In [ ]:
fig, ax = plt.subplots()

terrains = sorted(vulpi_pro.terrain.unique())
rhos = [vulpi_pro[vulpi_pro.terrain == terr].rho for terr in terrains]
labels = [lab.replace("_", " ").title() for lab in terrains]
cols = [PLOTCOLORS[lab.lower()] for lab in labels]

ax.hist(
    rhos,
    label=labels,
    color=cols,
    log=True,
    bins=10,
)
ax.legend(loc="center left", bbox_to_anchor=(1.05, 0.5))
ax.set_xlabel("Rotationnality $\\rho$")
ax.set_ylabel("Counts")
ticks = np.round(0.1 * np.arange(11), 1)
ax.set_xticks(ticks, ticks)
plt.show()